## Lab Assignment 2
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

### Data Preparation

#### Section 1

- (10pts) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

---

#### Feature Engineering
Based on results from the Mini-lab, we investigated possible features that will improve the model's ability to select *True Positives* or actual **Podium Finishers**.  In the mini-lab we added `Population_Prop` which is world populatio proportion for each represented country with the thought that larger countries would produce a higher proportion of **Podium Finishers**.  We also added the previously calclated `BMI` feature to the dataset.  A final addition for this assignment, was to reward prior medal winners when they compete in future Olympic Games.  This feature, `Previous_Medals`, is a binary feature that establishes an athlete as **Podium Finisher** caliber. 

In [1]:
import numpy as np
import pandas as pd
#Combine existing data with population data from:
#https://data.world/worldbank/total-population-per-country/file/POP_TOTAL_DS2_en_v2.csv
df = pd.read_csv('~/olympics/data/athletes_cleaned_merged.csv')
df_pop = pd.read_csv('~/olympics/data/POP_TOTAL.csv', encoding = "ISO-8859-1")
df_pop = df_pop[['Country Code', '2015']]
#Change all Russia NOCs
df['NOC'] = np.where(df['NOC'] == 'EUN', 'RUS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'URS', 'RUS', df['NOC'])
#Change all Australia NOCs
df['NOC'] = np.where(df['NOC'] == 'ANZ', 'AUS', df['NOC'])
#Change all German NOCs
df['NOC'] = np.where(df['NOC'] == 'FRG', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'GDR', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SAA', 'GER', df['NOC'])
#Change all Congo NOCs
df['NOC'] = np.where(df['NOC'] == 'CGO', 'COD', df['NOC'])
#Change all Czech NOCs
df['NOC'] = np.where(df['NOC'] == 'BOH', 'CZE', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'TCH', 'CZE', df['NOC'])
#Change all Yemen NOCs
df['NOC'] = np.where(df['NOC'] == 'YAR', 'YEM', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'YMD', 'YEM', df['NOC'])
#Change all Greek NOCs
df['NOC'] = np.where(df['NOC'] == 'CRT', 'GRE', df['NOC'])
#Change all Zimbabwe NOCs
df['NOC'] = np.where(df['NOC'] == 'RHO', 'ZIM', df['NOC'])
#Change all Malaysia NOCs
df['NOC'] = np.where(df['NOC'] == 'MAL', 'MAS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'NBO', 'MAS', df['NOC'])
#Change all Vietnam NOCs
df['NOC'] = np.where(df['NOC'] == 'VNM', 'VIE', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'WIF', 'TTO', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'UAR', 'SYR', df['NOC'])
#Change all Serbian NOCs
df['NOC'] = np.where(df['NOC'] == 'SCG', 'YUG', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SRB', 'YUG', df['NOC'])
#Change all Canada NOCs
df['NOC'] = np.where(df['NOC'] == 'NFL', 'CAN', df['NOC'])
df_pop.rename(index = str, columns = {'Country Code' : 'NOC', '2015': '2015 Population'}, inplace = True)
df_pop_merge = pd.merge(left = df, right = df_pop, how = 'left', on=['NOC'], left_index=False)
df_pop_merge.loc[df_pop_merge['Country'] == 'Taiwan', '2015 Population'] =  23485755
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Vincent', '2015 Population'] =  109643
df_pop_merge.loc[df_pop_merge['Country'] == 'Palestine', '2015 Population'] =  4817000
df_pop_merge.loc[df_pop_merge['Country'] == 'Cook Islands', '2015 Population'] =  17459
df_pop_merge.loc[df_pop_merge['Country'] == 'Eritrea', '2015 Population'] =  4846976
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Kitts', '2015 Population'] =  54821
df_pop_mergena = df_pop_merge[df_pop_merge['2015 Population'].isnull()]
#Drop the 106 observations not associated with a country 
df_pop_merge = df_pop_merge.dropna(how = 'any')

In [2]:
#Caluclate the population proportion for each country
df_pop = df_pop_merge.groupby(['Country'])['2015 Population'].mean().reset_index()
df_pop['Population_Prop'] = df_pop['2015 Population'].transform(lambda x: x / (x.sum()))

#Calculate BMI for each athlete
def calculate_bmi(df):
    return (df[1]/(df[0] * df[0])) * 10000
df_pop_merge['BMI'] = df_pop_merge[['Height', 'Weight']].apply(calculate_bmi, axis=1)

#Merge dataframes with Population Propotion information
df_pop_merge = pd.merge(left = df_pop_merge, right = df_pop, how = 'left', on=['Country'], 
                        left_index=False)
#Create Binary Medal, Season, and Sex Classes
df_pop_merge['Medal'] = np.where(df_pop_merge.Medal == 'No Medal', 0, 1)
df_pop_merge['Season'] = np.where(df_pop_merge.Season == 'Summer', 1, 0)
df_pop_merge['Sex'] = np.where(df_pop_merge.Sex == 'M', 1, 0)

In [3]:
%%time

prev_medals = df_pop_merge.groupby(['Name', 'Year'])['Medal'].sum()
prev_medals = prev_medals.reset_index()
prev_medals.columns = ['Name', 'Year', 'Medal_Count']
prev_medals['Previous_Medals'] = prev_medals['Medal_Count']

for i in range(2, len(prev_medals)):
    if prev_medals.iloc[i]['Name'] == prev_medals.iloc[i-1]['Name']:
        if prev_medals.iloc[i-1]['Medal_Count'] != 0:
            prev_medals['Previous_Medals'].iloc[i] = 1
        else:
            prev_medals['Previous_Medals'].iloc[i] = 0
    else:
        prev_medals['Previous_Medals'].iloc[i] = 0
print(prev_medals.head())

/Users/Steve/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                                        Name  Year  Medal_Count  \
0    Gabrielle Marie "Gabby" Adcock (White-)  2016            0   
1        Eleonora Margarida Josephina Scmitt  1948            0   
2                            Jean Hauptmanns  1912            0   
3    Luis ngel Fernando de los Santos Grossi  1948            0   
4    Luis ngel Fernando de los Santos Grossi  1952            0   

   Previous_Medals  
0                0  
1                0  
2                0  
3                0  
4                0  
CPU times: user 1h 19min 47s, sys: 26.6 s, total: 1h 20min 14s
Wall time: 1h 20min 19s


In [4]:
#Merge prev_medals into existing pop_merge dataset to add Previous_Medals feature
df_prev_medal_merge = pd.merge(left = df_pop_merge, right = prev_medals, how = 'left', on = ['Name', 'Year'])
df_pop_merge = df_prev_medal_merge[['Country', 'Sex', 'Age', 'Height', 'Weight', 'Year', 'Season', 'Sport', 'Event', 'BMI', 
                             'Population_Prop', 'Medal', 'Previous_Medals']]

#Write dataframe to .csv file in order to preserve the data and avoid Feature Engineering code in future iterations 
df_pop_merge.to_csv('~/olympics/data/pop_merge.csv')

In [5]:
#Create X and y vectors for sklearn pre-processing steps.
df = pd.read_csv('~/olympics/data/pop_merge.csv')

y = df['Medal']
X = df.drop(['Medal'], axis = 1)

print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266848 entries, 0 to 266847
Data columns (total 13 columns):
Unnamed: 0         266848 non-null int64
Country            266848 non-null object
Sex                266848 non-null int64
Age                266848 non-null int64
Height             266848 non-null int64
Weight             266848 non-null int64
Year               266848 non-null int64
Season             266848 non-null int64
Sport              266848 non-null object
Event              266848 non-null object
BMI                266848 non-null float64
Population_Prop    266848 non-null float64
Previous_Medals    266848 non-null int64
dtypes: float64(2), int64(8), object(3)
memory usage: 26.5+ MB
None


In [6]:
X = pd.get_dummies(X, columns = ['Country', 'Event', 'Sport'])
print(X.describe())
print(y.describe())

          Unnamed: 0            Sex            Age         Height  \
count  266848.000000  266848.000000  266848.000000  266848.000000   
mean   133423.500000       0.722786      25.330338     175.420948   
std     77032.526654       0.447624       5.801661       9.869066   
min         0.000000       0.000000      10.000000     127.000000   
25%     66711.750000       0.000000      21.000000     169.000000   
50%    133423.500000       1.000000      24.000000     176.000000   
75%    200135.250000       1.000000      28.000000     182.000000   
max    266847.000000       1.000000      72.000000     226.000000   

              Weight           Year         Season            BMI  \
count  266848.000000  266848.000000  266848.000000  266848.000000   
mean       71.017542    1979.016369       0.818185      22.904691   
std        13.330644      29.560519       0.385693       2.718862   
min        25.000000    1896.000000       0.000000       8.360954   
25%        62.000000    1960.0000

In [7]:
from sklearn.preprocessing import StandardScaler
#Standardize X
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

#### Section 2

- (5pts) Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

---

### Modeling and Evaluation

#### Section 1

- (10 pts) Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

---

#### Section 2

- (10pts) Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

---

In [8]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)

#### Section 3

- (20pts) Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

---

#### Task 1 (Classify Podium Finishers)

In [10]:
# Classifer Estimator created and edited from Professor Drew's NC school repository
def EvaluateClassifierEstimator2(classifierEstimator, X, y, cv):
    
    #Perform cross validation 
    from sklearn.model_selection import cross_val_predict
    predictions = cross_val_predict(classifierEstimator, X, y, cv=cv)
    
    #model evaluation 
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
    
    #pass true test set values and predictions to classification_report
    classReport = classification_report(y, predictions)
    confMat = confusion_matrix(y, predictions)
    acc = accuracy_score(y, predictions)

    
    print(classReport)
    print(confMat)
    print(acc)

In [ ]:
#Logisitic regression 10-fold cross-validation 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()


parameters = { 'penalty':['l2','l1']
              ,'C': [0.001, 0.01, 0.1, 1, 10, 100]
              ,'class_weight': ['balanced','none']
              ,'random_state': [0]
              ,'solver': ['sag', 'saga']
              ,'max_iter':[100,500]
             }

#Create a grid search object using the  
regGS = GridSearchCV(estimator=lr
                   , n_jobs=-1 #Use all possible cores to run jobs in parallel
                   , verbose=True
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring='f1')

#Perform hyperparameter search to find the best combination of parameters for our data
regGS.fit(X_std, Y)

In [ ]:
lr_clf = GS.best_estimator_

In [ ]:
EvaluateClassifierEstimator2(lr_clf, X_std, y, 10)

In [ ]:
lr_clf = lr_clf.fit(X_std, y)

In [ ]:
from sklearn.externals import joblib
gs_lr_model = 'gs_lr_model.joblib.pkl'
_ = joblib.dump(lr_clf, gs_lr_model, compress = 9)
#This will save the model in your working directory

In [ ]:
clf2 = joblib.load(gs_lr_model)
clf2

#### Section 4

- (10pts) Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

---

#### Section 5

- (10pts) Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

---

#### Section 6

- (10pts) Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

---

### Deployment

- (5pts) How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

---

### Exceptional Work

- (10pts) You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

---